In [1]:
cd ~/code/ctl/build/python

/Users/rhl/code/ctl/build/python


In [2]:
%matplotlib inline
import ctl
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.tri import Triangulation
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial import ConvexHull
def sphere_sample(N, dim=4):
    norm = np.random.normal
    normal_deviates = norm(size=(dim, N))
    radius = np.sqrt((normal_deviates**2).sum(axis=0))
    return (normal_deviates/radius).T

h = lambda l : [np.square(l[0]) + np.square(l[1])**2 - np.square(l[2])**2 - np.square(l[3])**2, 2*(l[0]*l[3] + l[1]*l[2]), 2*(l[1]*l[3]-l[0]*l[3])]

In [3]:
D = sphere_sample(1000)
f = h(D.T)

In [4]:
K = ctl.Simplicial_complex([ctl.Abstract_simplex(s.tolist()) for s in ConvexHull(D).simplices])

In [5]:
ctl.homology(K)

[1L, 0L, 0L, 1L]

In [6]:
np.median(f[0])

0.18812374707110741

In [7]:
print K.size()

28108


In [8]:
import line_profiler
import IPython
ip = IPython.get_ipython()
ip.define_magic('lprun', line_profiler.magic_lprun)

In [9]:
def compute_blowup_pieces(K, f, h1=.11, h2=.18):
    KX = ctl.Simplicial_complex()
    KY = ctl.Simplicial_complex()
    KI = ctl.Simplicial_complex()
    for (i, (s,d)) in enumerate(K):
        insX = False
        insY = False
        for vtx in s:
            inX = (f[0][vtx] > h1)
            inY = (f[0][vtx] < h2) 
            in_inter= inX and inY
            if not insX and inX:
                KX.insert_closed_cell(s)
                insX = True
            elif not insY and inY:
                KY.insert_closed_cell(s)
                insY = True
            if (in_inter):
                KI.insert_closed_cell(s)
                KX.insert_closed_cell(s)
                KY.insert_closed_cell(s)
                break
    return (KX,KY,KI)

In [12]:
(KX,KY,KI) = compute_blowup_pieces(K,f,.115,.2)
(X_bti, X_chains) = ctl.homology(KX, True)
(Y_bti, Y_chains) = ctl.homology(KY, True)
(I_bti, I_chains) = ctl.homology(KI, True)
print "H_*(KX)", X_bti
print "H_*(KY)", Y_bti
print "H_*(KI)", I_bti

H_*(KX) [1L, 2L, 0L, 0L]
H_*(KY) [1L, 1L, 0L, 0L]
H_*(KI) [1L, 19L, 1L, 0L]


In [ ]:
def plot_2skel(X, K):
    X -= X.mean(axis=0)
    triangles = np.array([list(s) for (s,d) in K if s.dimension() == 2])
    rad = np.linalg.norm(X, axis=1)
    zen = np.arccos(X[:,-1] / rad)
    azi = np.arctan2(X[:,1], X[:,0])
    tris = Triangulation(zen, azi)
    fig = plt.figure()
    ax  = fig.add_subplot(111, projection='3d')
    ax.plot_trisurf(X[:,0], X[:,1], X[:,2], triangles=triangles, cmap=plt.cm.bone)
    plt.show()

In [ ]:
plot_2skel(D, KX)

In [ ]:
plot_2skel(D, KY)

In [ ]:
plot_2skel(D, KI)

In [ ]:
ctl.homology(KX)

In [ ]:
ctl.homology(KY)

In [ ]:
ctl.homology(KI)

In [ ]:
Kcheck = ctl.Simplicial_complex()
for (s,d) in KX:
    Kcheck.insert_open_cell(s)
for (s,d) in KY:
    Kcheck.insert_open_cell(s)
for (s,d) in KI:
    Kcheck.insert_open_cell(s)
print Kcheck.is_closed()
ctl.homology(Kcheck)